In [1]:
# import OS module
import os
import numpy as np
from matplotlib.pyplot import *
from scipy.misc import *
from skimage import color
from skimage import io
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Get the list of all files and directories
path = "./images/"
dir_list = os.listdir(path)

In [3]:
data = np.zeros((1, 504, 504))
data = data[1:]
data.shape

(0, 504, 504)

In [4]:
for file in dir_list:
    img = color.rgb2gray(imread(path + file)[29: , :, :])
    data = np.concatenate((data,np.reshape(img, (1, 504, 504)) ))
data.shape

(720, 504, 504)

In [5]:
labels = open("labels.txt", "r")
labels = labels.read()
labels = labels.split('\n')

In [6]:
hour_labels = np.zeros((1))[1:]
min_labels = np.zeros((1))[1:]

for lbl in labels:
    time = lbl.split(":")
    if len(time) == 2:
        hour_labels = np.append(hour_labels, time[0])
        min_labels = np.append(min_labels, time[1])


In [7]:
p = np.random.permutation(len(data))
X = data[p, :].astype(float)
h = hour_labels[p].astype(float)
m = min_labels[p].astype(float)

In [8]:
train_data , test_data = X[:504, :], X[504 :, :]
train_h_lbls, test_h_lbls = h[:504], h[504 :]
train_m_lbls, test_m_lbls = m[:504], m[504 :]

In [9]:
inputs = keras.Input(shape=(504, 504, 1))
x = layers.Conv2D(filters=8, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(13, activation="softmax")(x)
model_hours = keras.Model(inputs=inputs, outputs=outputs)

In [10]:
model_hours.compile(optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
model_hours.fit(train_data, train_h_lbls, epochs=10, batch_size=64)

Epoch 1/10
8/8 [==============================] - 19s 2s/step - loss: 19.7450 - accuracy: 0.0853
Epoch 2/10
8/8 [==============================] - 18s 2s/step - loss: 2.6063 - accuracy: 0.1190
Epoch 3/10
8/8 [==============================] - 18s 2s/step - loss: 1.7768 - accuracy: 0.4266
Epoch 4/10
8/8 [==============================] - 18s 2s/step - loss: 0.7279 - accuracy: 0.7917
Epoch 5/10
8/8 [==============================] - 18s 2s/step - loss: 0.2415 - accuracy: 0.9405
Epoch 6/10
8/8 [==============================] - 18s 2s/step - loss: 0.1153 - accuracy: 0.9722
Epoch 7/10
8/8 [==============================] - 18s 2s/step - loss: 0.0502 - accuracy: 0.9841
Epoch 8/10
8/8 [==============================] - 18s 2s/step - loss: 0.0463 - accuracy: 0.9821
Epoch 9/10
8/8 [==============================] - 18s 2s/step - loss: 0.0475 - accuracy: 0.9861
Epoch 10/10
8/8 [==============================] - 18s 2s/step - loss: 0.0231 - accuracy: 0.9960


In [11]:
test_loss, test_acc = model_hours.evaluate(test_data, test_h_lbls)
print(f"Test accuracy: {test_acc:.3f}")

7/7 [==============================] - 2s 284ms/step - loss: 0.2290 - accuracy: 0.9120
Test accuracy: 0.912


In [12]:
inputs = keras.Input(shape=(504, 504, 1))
x = layers.Conv2D(filters=8, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=16, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(60, activation="softmax")(x)
model_minutes = keras.Model(inputs=inputs, outputs=outputs)

In [13]:
model_minutes.compile(optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
model_minutes.fit(train_data, train_m_lbls, epochs=10, batch_size=64)

Epoch 1/10
8/8 [==============================] - 21s 2s/step - loss: 19.8768 - accuracy: 0.0218
Epoch 2/10
8/8 [==============================] - 19s 2s/step - loss: 3.9709 - accuracy: 0.0873
Epoch 3/10
8/8 [==============================] - 19s 2s/step - loss: 2.1844 - accuracy: 0.4821
Epoch 4/10
8/8 [==============================] - 19s 2s/step - loss: 0.4747 - accuracy: 0.9385
Epoch 5/10
8/8 [==============================] - 19s 2s/step - loss: 0.1512 - accuracy: 0.9742
Epoch 6/10
8/8 [==============================] - 19s 2s/step - loss: 0.0935 - accuracy: 0.9762
Epoch 7/10
8/8 [==============================] - 20s 3s/step - loss: 0.0770 - accuracy: 0.9821
Epoch 8/10
8/8 [==============================] - 20s 2s/step - loss: 0.0676 - accuracy: 0.9841
Epoch 9/10
8/8 [==============================] - 19s 2s/step - loss: 0.0606 - accuracy: 0.9821
Epoch 10/10
8/8 [==============================] - 19s 2s/step - loss: 0.0552 - accuracy: 0.9861


In [14]:
test_loss, test_acc = model_minutes.evaluate(test_data, test_m_lbls)
print(f"Test accuracy: {test_acc:.3f}")

7/7 [==============================] - 3s 295ms/step - loss: 0.2991 - accuracy: 0.9444
Test accuracy: 0.944


In [15]:
def predict_time(image):
    hrs = model_hours.predict(np.reshape(image, (1,504,504, 1)), verbose = 0).argmax(axis=1)
    mins =  model_minutes.predict(np.reshape(image, (1,504,504, 1)), verbose = 0).argmax(axis=1)
    return str(hrs[0]).zfill(2) + ":" + str(mins[0]).zfill(2)

In [17]:
test_correct_preds = 0
for i in range(len(test_data)):
    t = predict_time(test_data[i])
    lbl = str(int(test_h_lbls[i])).zfill(2) + ":" + str(int(test_m_lbls[i])).zfill(2)
    #print( t + "\t" + str(int(test_h_lbls[i])).zfill(2) + ":" + str(int(test_m_lbls[i])).zfill(2))
    test_correct_preds += (t == lbl)
test_accuracy = test_correct_preds / len(test_data)

In [18]:
test_accuracy

0.8703703703703703

In [19]:
train_correct_preds = 0
for i in range(len(train_data)):
    t = predict_time(train_data[i])
    lbl = str(int(train_h_lbls[i])).zfill(2) + ":" + str(int(train_m_lbls[i])).zfill(2)
    #print( t + "\t" + str(int(test_h_lbls[i])).zfill(2) + ":" + str(int(test_m_lbls[i])).zfill(2))
    train_correct_preds += (t == lbl)
train_accuracy = train_correct_preds / len(train_data)

In [20]:
train_accuracy

0.9900793650793651